# Export Growth Prediction

## Model 1: Multivariate Linear Regression with Categorical Variable

Formula:

$$(Ex - \overline{Ex_{c}}) =
\theta_{0} +
\theta_{1}(\log GDP - \overline{\log GDP_c}) +
\theta_{2}(GDP_{capita} - \overline{{GDP_{capita}}_c}) +
\theta_{3}(g_{im} - \overline{{g_{im}}_c}) + \epsilon$$

where $\overline{Ex_c}$, $\overline{\log GDP_j}$, $\overline{{GDP_{capita}}_c}$, and $\overline{{g_{im}}_c}$ are means over all years for country $c$ of export value, GDP, GDP per capita, and import growth, respectively. Also known as the fixed effects model.

## 1.1 Bad Practice vs. Standard Approach

* Bad:
    - Training: 1998-2013 (16 years)
    - Test: 2013
* Standard:
    - Training: 1997-2012 (16 years)
    - Test: 2013

### Comparing Performance

### Learning Curve: Are More Data Likely to Help?

## 1.2 Training with More Recent Data

- Training: 1997-2015 (19 years)
- Test: 2016

### Evaluation

### Error Analysis

### Prediction